<a href="https://colab.research.google.com/github/RaghunandanVenkatesh/LearningToSee/blob/master/RL_Hvac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline

In [ ]:
#inputs for plant
T_oat = 30.0
PWM_front_box = 30.0
T_enginewater_set = 80.0
POS_fresh_air_flap = 40.0
value = 0
dt = 1 # sample time

In [ ]:
# initialization 
water_val_pos_list = np.array([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
A_list = np.array([0, 0.6011, .61, 0.6, 1.88, 1.88, 2.1, 2.1, 2.1, 2.5, 3.5])
lookup = UnivariateSpline(water_val_pos_list, A_list, k=1, s=0.0)
T_enginewater = T_oat
T_fap = T_oat
hA_screen = 0.0007
hA_shell = 0.0029
convfactor = 0.0016
mcp_shell = 0.83
T_shell = T_oat
T_set = 24


# Engine water temp

In [ ]:
T_enginewater = T_oat + ( T_enginewater_set + T_oat - T_enginewater) * (1 - np.exp(-value)) 
value = np.min([value+0.002, 5]) 

# Airoutlet temperature

In [ ]:
T_air_in = T_oat * POS_fresh_air_flap/100 + T_fap * (1 - POS_fresh_air_flap/100)
A = lookup(POS_fresh_air_flap)
eff = 1 - np.exp(-POS_fresh_air_flap*A/100)
T_airout = T_air_in + (T_enginewater - T_air_in) * eff

# room temperature 

In [ ]:
d_T_fap = hA_screen * (T_oat - T_fap) + hA_shell * (T_shell - T_fap)
d_T_fap += PWM_front_box * convfactor * 0.718 * (T_airout - T_fap)
T_fap = d_T_fap * dt + T_fap
T_shell = T_shell - hA_shell * (T_shell - T_fap)/mcp_shell

In [ ]:
class Environment:
  def __init__(self, T_oat, PWM_front_box, T_enginewater_set, POS_fresh_air_flap, T_set ):
      self.T_oat = T_oat
      self.PWM_front_box = PWM_front_box
      self.T_enginewater_set = T_enginewater_set
      self.POS_fresh_air_flap = POS_fresh_air_flap
      self.value = 0
      self.dt = 1 # sample time
      self.T_set = T_set
      # initialization 
      water_val_pos_list = np.array([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
      A_list = np.array([0, 0.6011, .61, 0.6, 1.88, 1.88, 2.1, 2.1, 2.1, 2.5, 3.5])
      self.lookup = UnivariateSpline(water_val_pos_list, A_list, k=1, s=0.0)
      self.T_enginewater = T_oat
      self.T_fap = T_oat
      self.hA_screen = 0.0007
      self.hA_shell = 0.0029
      self.convfactor = 0.0016
      self.mcp_shell = 0.83
      self.T_shell = T_oat

  def step(self):
      # engine water temp
      self.T_enginewater = self.T_oat + ( self.T_enginewater_set + self.T_oat - self.T_enginewater) * (1 - np.exp(-self.value)) 
      self.value = np.min([self.value+0.002, 5])
      # air outlet temp
      T_air_in = self.T_oat * self.POS_fresh_air_flap/100 + self.T_fap * (1 - self.POS_fresh_air_flap/100)
      A = self.lookup(self.POS_fresh_air_flap)
      eff = 1 - np.exp(-self.POS_fresh_air_flap*A/100)
      T_airout = T_air_in + (self.T_enginewater - T_air_in) * eff 
      # room temperature
      d_T_fap = self.hA_screen * (self.T_oat - self.T_fap) + self.hA_shell * (self.T_shell - self.T_fap)
      d_T_fap += self.PWM_front_box * self.convfactor * 0.718 * (T_airout - self.T_fap)
      self.T_fap = d_T_fap * dt + self.T_fap
      self.T_shell = self.T_shell - self.hA_shell * (self.T_shell - self.T_fap)/self.mcp_shell    

      #reward
      reward = -(self.T_set - self.T_fap)**2
      observation = [self.T_fap]
      info = False
      return observation, reward, info

  def reset(self):
    self.T_shell = T_oat
    self.T_fap = T_oat
    self.T_enginewater = T_oat
    observation = [self.T_fap]
    return observation                     

In [ ]:
upper_temp_fap = 60
min_temp_fap = -40
env = Environment( T_oat, PWM_front_box, T_enginewater_set, POS_fresh_air_flap, T_set )

In [ ]:
env.step()

([30.0], -36.0, False)

In [1]:
import math
import gym
from gym import spaces, logger
from gym.utils import seeding
import numpy as np

In [9]:
class HvacPlantEnv(gym.Env):
  '''
  Description:
  this environment is used as a digital bus temperature model. It has 2 inputs.
  1st input is a outside air temperature (T_oat) from sensor
  2nd input is a set temperature (T_set) from driver

  Observation:
  Type: Box(1)
  Num     Observation               Min                     Max
  0       T_fap                     -40                     80

  Actions:
  Type: Tuple(2)
  Num     Observation                            Type
  0       value of POS_fresh_air_flap            Discrete(10)
  1       value of PWM_front_box                 Discrete(100)

  Reward:
  reward is a squared error loss between T_set and T_fap           

  Starting_state:
  all the initial temperature values are equal to T_aot

  Episode termination:
  When T_fap is equal to T_set for atleast 10000 cycles
  '''

  def __init__(self):
    self.dt = 1 # sample time
    # initialization 
    water_val_pos_list = np.array([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
    A_list = np.array([0, 0.6011, .61, 0.6, 1.88, 1.88, 2.1, 2.1, 2.1, 2.5, 3.5])
    self.lookup = UnivariateSpline(water_val_pos_list, A_list, k=1, s=0.0)
    self.hA_screen = 0.0007
    self.hA_shell = 0.0029
    self.convfactor = 0.0016
    self.mcp_shell = 0.83
    self.T_oat = 0
    self.T_enginewater_set = 0
    self.action_space = spaces.Tuple((
                            spaces.Discrete(10),
                            spaces.Discrete(100)))
    self.observation_space = spaces.Box(-40, 80, shape=(1,) ,dtype=np.float32)


    self.state = None

    self.steps_beyond_done = None

    def step(self, action):
        T_shell, T_fap, T_enginewater, value = self.state
        POS_fresh_air_flap = action[0]*10
        PWM_front_box = action[1]
        # engine water temp
        T_enginewater = self.T_oat + ( self.T_enginewater_set + self.T_oat - T_enginewater) * (1 - np.exp(-value)) 
        value = np.min([value+0.002, 5])
        # air outlet temp
        T_air_in = self.T_oat * POS_fresh_air_flap/100 + T_fap * (1 - POS_fresh_air_flap/100)
        A = self.lookup(POS_fresh_air_flap)
        eff = 1 - np.exp(-POS_fresh_air_flap*A/100)
        T_airout = T_air_in + (T_enginewater - T_air_in) * eff 
        # room temperature
        d_T_fap = self.hA_screen * (self.T_oat - T_fap) + self.hA_shell * (self.T_shell - T_fap)
        d_T_fap += PWM_front_box * self.convfactor * 0.718 * (T_airout - T_fap)
        T_fap = d_T_fap * dt + T_fap
        T_shell = T_shell - self.hA_shell * (T_shell - T_fap)/self.mcp_shell 
        self.state = (T_shell, T_fap, T_enginewater, value)   

        #reward
        done  = self.T_set == T_fap
        if not done:
          reward = (self.T_set - T_fap)**2
        elif self.steps_beyond_done is  None:
          self.steps_beyond_done = 0
          reward = (self.T_set - T_fap)**2
        else:
          if self.steps_beyond_done == 0:
                print(
                    "You are calling 'step()' even though this "
                    "environment has already returned done = True. You "
                    "should always call 'reset()' once you receive 'done = "
                    "True' -- any further steps are undefined behavior."
                )
          self.steps_beyond_done += 1
          reward = 0.0
        return np.array(self.state), reward, done, {} 

    def reset(self, T_oat, T_enginewater_set):
      self.T_oat = T_oat
      self.T_enginewater_set = T_enginewater
      self.state = [T_oat, T_oat, T_oat, 0]
      self.steps_beyond_done = None
      return np.array(self.state)

In [11]:
action = spaces.Tuple((spaces.Discrete(10), spaces.Discrete(100)))
env_h = HvacPlantEnv()
env_h.reset(24,40)
env_h.step(action)

TypeError: ignored